### Swaption Valuation using Black76
We will price a Swaption using Black76 model.\
Let us consider that there is a 1x2 swaption. The swap has semi annual frequency. Assume a notional of $1M.\
At Time T0, the yield curve is as below:

Tenor 0.5 = 5.00% \
Tenor 1.0 = 6.00% \
Tenor 1.5 = 6.75% \
Tenor 2.0 = 7.25% \
Tenor 2.5 = 7.50% \
Tenor 3.0 = 7.60% 

Calculate the Swaption price using Black76 Model assuming a constant vol of 2% and a fixed rate of 8%

In [115]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [137]:
def swapRateCalc(YieldMatrix,Alpha,Beta,Tau):
    #Compute the Swap rate at time t=0 as (P@Alpha - P@Beta)/(Tau * Sum of P from Tenor(start payment to end payment))
    P0Alpha = YieldMatrix.loc[YieldMatrix['Tenor']==Alpha,'P'].squeeze()
    P0Beta = YieldMatrix.loc[YieldMatrix['Tenor']==Beta,'P'].squeeze()
    startPayment = YieldMatrix['Tenor'].tolist().index(Alpha+Tau)
    EndPayment = YieldMatrix['Tenor'].tolist().index(Beta)+1
    SumP0 = np.sum(YieldMatrix['P'][startPayment:EndPayment])
    S0_AlphaBeta = (P0Alpha - P0Beta)/(Tau * SumP0)
    return(S0_AlphaBeta)

In [141]:
def swapValuation(YieldMatrix,Alpha,Beta,Tau,N,S0_AlphaBeta,k):
    SwapValue_TAlpha = 0
    YieldMatrix['PAlpha'] = np.exp(-YieldMatrix['Yield'] * (YieldMatrix['Tenor']-Alpha)) #Discounting to Time at Alpha
    YieldMatrix['PAlpha'][YieldMatrix['Tenor']<Alpha] = 0 #Value of Bond with Tenor less than Alpha is 0
    
    startPayment = YieldMatrix['Tenor'].tolist().index(Alpha+Tau)
    EndPayment = YieldMatrix['Tenor'].tolist().index(Beta)+1
    
    #Value of a swap is given as (swapRate - fixedRate) * Sum of Present Value of payments
    SumPAlpha = np.sum(YieldMatrix['PAlpha'][startPayment:EndPayment]) * YieldMatrix['P'][YieldMatrix['Tenor']==Alpha]
    SwapValue_TAlpha = (S0_AlphaBeta - k) * SumPAlpha * N * Tau
    return(SwapValue_TAlpha)
    

In [142]:
def Bl76_SwaptionValuation(YieldMatrix,Alpha,Beta,Tau,N,S0_AlphaBeta,k,Vol):
    SwaptionValue = 0
    YieldMatrix['PAlpha'] = np.exp(-YieldMatrix['Yield'] * (YieldMatrix['Tenor']-Alpha)) #Discounting to Time at Alpha
    YieldMatrix['PAlpha'][YieldMatrix['Tenor']<Alpha] = 0 #Value of Bond with Tenor less than Alpha is 0
    
    startPayment = YieldMatrix['Tenor'].tolist().index(Alpha+Tau)
    EndPayment = YieldMatrix['Tenor'].tolist().index(Beta)+1
    
    d1 = (np.log(S0_AlphaBeta/k) + (0.5 * Vol**2) * Alpha)/(Vol * np.sqrt(Alpha))
    d2 = d1 - Vol * np.sqrt(Alpha)
    #Value of Swaption is given by Sum of Present Value of payments * max(swapRate - k,0)
    SumPAlpha = np.sum(YieldMatrix['PAlpha'][startPayment:EndPayment]) * YieldMatrix['P'][YieldMatrix['Tenor']==Alpha]
    SwaptionValue = SumPAlpha * N * Tau * (S0_AlphaBeta * norm.cdf(d1) - k * norm.cdf(d2))
    return(SwaptionValue)
    

In [144]:
Alpha = 1
Beta = 3
Vol = 0.02
Tau = 0.5 #Semiannual frequency
N = 1000000 #$1 million Notional
f = (Beta-Alpha)/Tau #Number of payments
k = 0.08 #Assume Fixed Rate is 8%

YieldMatrix = pd.DataFrame([[0.5,0.05],[1.0,0.06],[1.5,0.0675],[2.0,0.0725],[2.5,0.075],[3.0,0.076]])
YieldMatrix = YieldMatrix.rename(columns = {0:'Tenor',1:'Yield'})

#Price of a Zero Coupon Bond at time t=0 for all the tenors
YieldMatrix['P'] = np.exp(-YieldMatrix['Yield'] * YieldMatrix['Tenor'])

#Swap Rate
S0_AlphaBeta = swapRateCalc(YieldMatrix,Alpha,Beta,Tau)
print("The Swap Rate is :" , np.round(S0_AlphaBeta * 100,4),"%")

#Swap Valuation
#Valuation of swaption does not require the valuation of the swap as a parameter
#This is just a convenient segway to compute the value of the swap as well
SwapValue_TAlpha = swapValuation(YieldMatrix,Alpha,Beta,Tau,N,S0_AlphaBeta,k)

#Swaption Valuation
SwaptionValue = Bl76_SwaptionValuation(YieldMatrix,Alpha,Beta,Tau,N,S0_AlphaBeta,k,Vol)
print("The value of the swaption is :" , np.round(SwaptionValue,4).squeeze())


The Swap Rate is : 8.5825 %
The value of the swaption is : 10010.5181
